In [1]:
import torch
import torch.nn as nn


net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)



tensor([[-0.0267, -0.1398, -0.1535,  0.2704, -0.1317, -0.0803,  0.1115, -0.1180,
         -0.0575,  0.0261],
        [-0.1520, -0.0102, -0.0867,  0.2081,  0.0496, -0.0767,  0.1521,  0.0031,
          0.0163, -0.0364]], grad_fn=<AddmmBackward>)

In [2]:
net.__call__(X)

tensor([[-0.0267, -0.1398, -0.1535,  0.2704, -0.1317, -0.0803,  0.1115, -0.1180,
         -0.0575,  0.0261],
        [-0.1520, -0.0102, -0.0867,  0.2081,  0.0496, -0.0767,  0.1521,  0.0031,
          0.0163, -0.0364]], grad_fn=<AddmmBackward>)

In [3]:
from torch.nn import functional as F
class mlp(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256, 10)
    
    def forward(self,X):
        ou = self.hidden(X)
        ou = F.relu(ou)
        return self.out(ou)
        
        

In [4]:
net = mlp()

net(X)

tensor([[-0.0974, -0.0831, -0.0361,  0.2019, -0.0806,  0.1315, -0.0562,  0.0412,
          0.1765, -0.1411],
        [-0.0278, -0.1247, -0.1779,  0.1820, -0.0789,  0.0394,  0.0219,  0.1239,
          0.1645, -0.1352]], grad_fn=<AddmmBackward>)

In [9]:
class Sequential_mlp(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[idx] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
        
    

In [12]:
net =  Sequential_mlp(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

In [15]:
net(X), net(X).shape

(tensor([[-0.1608,  0.0721, -0.1787,  0.1526,  0.0990,  0.1479, -0.0612, -0.0483,
           0.1327,  0.2173],
         [-0.0300,  0.1611, -0.1256,  0.1131,  0.0620,  0.1523, -0.0266,  0.0035,
           0.0300,  0.1187]], grad_fn=<AddmmBackward>),
 torch.Size([2, 10]))

In [16]:
# using constant parameters
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.Linear = nn.Linear(20,20)
    
    def forward(self, X):
        X = self.Linear(X)
        
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        
        X = self.Linear(X)
        
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [17]:
net = FixedHiddenMLP()

In [18]:
net(X)

tensor(-0.0882, grad_fn=<SumBackward0>)

In [19]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(), nn.Linear(64,32), nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self, X):
        return self.linear(self.net(X))
        

In [20]:
net = nn.Sequential(NestMLP(), nn.Linear(16,20), FixedHiddenMLP())
net(X)

tensor(-0.0221, grad_fn=<SumBackward0>)

Exercises
1. What kinds of problems will occur if you change MySequential to store blocks in a Python
list?

* Whats MySequential

2. Implement a block that takes two blocks as an argument, say net1 and net2 and returns
the concatenated output of both networks in the forward propagation. This is also called a
parallel block.

* okayy

3. Assume that you want to concatenate multiple instances of the same network. Implement
a factory function that generates multiple instances of the same block and build a larger
network from it.

* hmm

In [55]:
net1 = NestMLP()
net2 = NestMLP()

In [56]:
net1(X),net2(X)

(tensor([[-0.2302, -0.0296, -0.0628,  0.1758, -0.0982, -0.2383,  0.1589, -0.0850,
           0.0350,  0.0720,  0.0218, -0.0485,  0.1184,  0.1400,  0.1794,  0.0982],
         [-0.2164, -0.0383, -0.0141,  0.1605, -0.1239, -0.2609,  0.2106, -0.0793,
           0.0125,  0.0606, -0.0071, -0.0718,  0.1286,  0.1397,  0.1545,  0.0735]],
        grad_fn=<AddmmBackward>),
 tensor([[-0.0945,  0.2564,  0.0341, -0.1094, -0.1193, -0.0155,  0.1945,  0.0521,
          -0.0302,  0.1418, -0.0156, -0.0660, -0.1055,  0.1927, -0.0262, -0.0785],
         [-0.0649,  0.1984, -0.0085, -0.1119, -0.1481,  0.0093,  0.2208,  0.0252,
          -0.0319,  0.0775, -0.0511, -0.0918, -0.0681,  0.1275, -0.0444, -0.0251]],
        grad_fn=<AddmmBackward>))

In [58]:
#2

class parallel_mlp(nn.Module):
    def __init__(self, block1, block2):
        super().__init__()
        self.block1 = block1
        self.block2 = block2

    def forward(self, X):
        first = self.block1(X)
        second = self.block2(X)
        print(first, second)
        return torch.cat((first, second))

In [59]:
net = parallel_mlp(net1, net2)

In [60]:
net(X)

tensor([[-0.2302, -0.0296, -0.0628,  0.1758, -0.0982, -0.2383,  0.1589, -0.0850,
          0.0350,  0.0720,  0.0218, -0.0485,  0.1184,  0.1400,  0.1794,  0.0982],
        [-0.2164, -0.0383, -0.0141,  0.1605, -0.1239, -0.2609,  0.2106, -0.0793,
          0.0125,  0.0606, -0.0071, -0.0718,  0.1286,  0.1397,  0.1545,  0.0735]],
       grad_fn=<AddmmBackward>) tensor([[-0.0945,  0.2564,  0.0341, -0.1094, -0.1193, -0.0155,  0.1945,  0.0521,
         -0.0302,  0.1418, -0.0156, -0.0660, -0.1055,  0.1927, -0.0262, -0.0785],
        [-0.0649,  0.1984, -0.0085, -0.1119, -0.1481,  0.0093,  0.2208,  0.0252,
         -0.0319,  0.0775, -0.0511, -0.0918, -0.0681,  0.1275, -0.0444, -0.0251]],
       grad_fn=<AddmmBackward>)


tensor([[-0.2302, -0.0296, -0.0628,  0.1758, -0.0982, -0.2383,  0.1589, -0.0850,
          0.0350,  0.0720,  0.0218, -0.0485,  0.1184,  0.1400,  0.1794,  0.0982],
        [-0.2164, -0.0383, -0.0141,  0.1605, -0.1239, -0.2609,  0.2106, -0.0793,
          0.0125,  0.0606, -0.0071, -0.0718,  0.1286,  0.1397,  0.1545,  0.0735],
        [-0.0945,  0.2564,  0.0341, -0.1094, -0.1193, -0.0155,  0.1945,  0.0521,
         -0.0302,  0.1418, -0.0156, -0.0660, -0.1055,  0.1927, -0.0262, -0.0785],
        [-0.0649,  0.1984, -0.0085, -0.1119, -0.1481,  0.0093,  0.2208,  0.0252,
         -0.0319,  0.0775, -0.0511, -0.0918, -0.0681,  0.1275, -0.0444, -0.0251]],
       grad_fn=<CatBackward>)

In [67]:
class hydra(nn.Module):
    def __init__(self, module, mutation_number):
        super().__init__()
        self.block = module
        self.m_n = mutation_number
        self.linear = nn.Linear(16, 20)
    
    def forward(self, X):
        for i in range(self.m_n):
            X = self.block(X)
            X = self.linear(X) 
        
        return X

In [68]:
net = hydra(net1, 3)

In [69]:
net(X)

tensor([[-0.0164, -0.2997, -0.1305,  0.1128, -0.0724, -0.1320, -0.1583, -0.1120,
         -0.1174,  0.0622,  0.0573,  0.2024,  0.0312,  0.1781,  0.2409, -0.0051,
          0.2348, -0.1194,  0.1917, -0.0742],
        [-0.0163, -0.2997, -0.1305,  0.1127, -0.0723, -0.1321, -0.1583, -0.1121,
         -0.1174,  0.0622,  0.0572,  0.2024,  0.0312,  0.1781,  0.2409, -0.0051,
          0.2348, -0.1193,  0.1917, -0.0742]], grad_fn=<AddmmBackward>)